In [ ]:
! wget 'https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_cats.txt'

--2023-12-29 03:00:22--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_cats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7400 (7.2K) [text/plain]
Saving to: ‘dev_cats.txt’

dev_cats.txt        100%[===================>]   7.23K  --.-KB/s    in 0s      

2023-12-29 03:00:23 (57.6 MB/s) - ‘dev_cats.txt’ saved [7400/7400]



### 1. Подгружаем обёртку NER модели из репозитория

In [ ]:
%%capture
! wget https://raw.githubusercontent.com/PhilBurub/Project_NLP/main/NER/NER_model.py
#! wget https://raw.githubusercontent.com/PhilBurub/Project_NLP/main/NER/requirements.txt
#! pip install -r requirements.txt
! pip install wget

In [ ]:
from NER_model import NER, Vocab
import torch
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import sent_tokenize
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 2. Применяем модель

In [ ]:
weights = 'https://github.com/PhilBurub/Project_NLP/raw/main/NER/model.pth'
tags = 'https://raw.githubusercontent.com/PhilBurub/Project_NLP/main/NER/tag_vocab.json'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ner_model = NER(weights, tags, device)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

In [ ]:
texts = pd.read_csv('https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt',
                    header=None, delimiter='\t', index_col=0, names=['text'])

In [ ]:
texts_li = texts['text'].tolist()
ids_li = texts.index.tolist()

In [ ]:
out_ner = pd.DataFrame(ner_model.get_ne(texts_li, ids_li), columns=['id', 'entity',	'start',	'end',	'aspect']).set_index('id')
out_ner = out_ner[out_ner['entity'] != '']
out_ner['sentiment'] = 0

In [ ]:
out_ner

,entity,start,end,aspect,sentiment
id,,,,,
13823,проводила к столу,188,205,Service,0
13823,официантка,242,252,Service,0
13823,ресторан,337,345,Whole,0
13823,заведения,431,440,Whole,0
13823,беседой,453,460,Service,0
...,...,...,...,...,...
11770,ресторане,117,126,Whole,0
11770,Муж,156,159,Food,0
11770,блюда,267,272,Food,0


### 4. Классификатор 1

#### 4.1. Препроцессинг

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import warnings
from copy import deepcopy
from collections import defaultdict
from transformers import BertModel, BertTokenizer, AdamW

rubert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny-sentiment-balanced").to(device)
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny-sentiment-balanced", device=device)

In [ ]:
def boudaries(id_, text):
  out = []
  new_text = text
  cur = 0
  for sent in sent_tokenize(text):
    start = new_text.find(sent)
    end = start + len(sent)
    out.append((id_, sent, cur + start, cur + end))
    cur += end
    new_text = new_text[end:]
  return pd.DataFrame(out, columns=['id', 'text', 'start', 'end'])

def create_mask(tokens, asp_tokens):
  mask_li = []
  for token in tokens:
    if token in asp_tokens:
      mask_li.append(1)
    else:
      mask_li.append(0)

  return mask_li

class ContextDataset(Dataset):
  def __init__(self, texts, outputs):

    new = pd.DataFrame()
    for id_, text in texts.iterrows():
      new = pd.concat((new, boudaries(id_, text['text'])))

    out = []
    for text_id, row in outputs.iterrows():
      slice_ = new[(new['id'] == text_id) & (new['start'] <= row['start']) & (new['end'] >= row['end'])]
      if len(slice_) == 0:
        #print(row, '0')
        continue
      if len(slice_) > 1:
        #print(row, '>1')
        continue
      out.append((text_id, row['entity'], row['aspect'], row['start'], row['end'], slice_['text'].item(), row['sentiment']))
    self.contexts = pd.DataFrame(out, columns=['id', 'input', 'aspect', 'start', 'end', 'context', 'sentiment'])

    self.map_sent = {0: -1, 'negative': 0, 'neutral': 1, 'positive': 2,
                     'both': 3}
    self.map_asp = {0: -1, 'Food': 0, 'Interior': 1, 'Price': 2,
                    'Service': 3, 'Whole': 4}

  def __len__(self):
    return len(self.contexts)

  def __getitem__(self, idx):

    row = self.contexts.loc[idx]
    text = row['context']
    tokens = tokenizer.encode_plus(text, max_length=150, add_special_tokens=True, padding='max_length', truncation=True, return_tensors='pt', return_token_type_ids=False)

    aspect_word = row['input']
    aspect_token_ids = tokenizer.encode(aspect_word, add_special_tokens=False, return_tensors='pt')
    asp_tokens = aspect_token_ids[0]
    tokens_list = tokens['input_ids'].tolist()[0]

    attention_mask = create_mask(tokens_list, asp_tokens)

    sentiment = [self.map_sent.get(row['sentiment'])]

    return {'review_text': text, 'input_ids': tokens['input_ids'], 'attention_mask': attention_mask, 'targets': torch.tensor(sentiment, dtype = torch.long).squeeze()}

####4.2. Загрузка модели

Загрузить в файлы чекпойнт модели *best_model_state.bin*

In [ ]:
!wget 'https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt'

--2023-12-29 02:43:32--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57508 (56K) [text/plain]
Saving to: ‘dev_aspects.txt.1’

dev_aspects.txt.1   100%[===================>]  56.16K  --.-KB/s    in 0.003s  

2023-12-29 02:43:32 (21.8 MB/s) - ‘dev_aspects.txt.1’ saved [57508/57508]



In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained("cointegrated/rubert-tiny-sentiment-balanced")
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    ).pooler_output

    output = self.drop(pooled_output)

    return self.out(output)


def get_predictions(model, data_loader):
  model = model.eval()

  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:
      texts = d["review_text"]
      input_ids = torch.squeeze(d['input_ids']).to(device)
      attention_mask = torch.stack(d['attention_mask']).transpose(0, 1).to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )

      preds = torch.argmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values


In [ ]:
def classify_with_ner(texts):
  texts_li = texts['text'].tolist()
  ids_li = texts.index.tolist()

  out_ner = pd.DataFrame(ner_model.get_ne(texts_li, ids_li), columns=['id', 'entity',	'start',	'end',	'aspect']).set_index('id')
  out_ner = out_ner[out_ner['entity'] != '']
  out_ner['sentiment'] = 0

  dev_set = ContextDataset(texts, out_ner)

  model = SentimentClassifier(4)
  model.load_state_dict(torch.load('best_model_state.bin'))
  model.eval()

  model.to(device)

  dev_data_loader = DataLoader(dev_set, batch_size=16, num_workers=4)
  y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model, dev_data_loader)

  df_pred = dev_set.contexts.copy()
  df_pred['sentiment'] = list(y_pred.tolist())
  df_pred['sentiment'] = df_pred['sentiment'].map({0:'negative', 1:'neutral', 2: 'positive', 3:'both'})

  return df_pred


texts = pd.read_csv('https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt',
                    header=None, delimiter='\t', index_col=0, names=['text'])

out = classify_with_ner(texts)
out.to_csv('dev_pred_aspects.txt', sep = '\t', header=False, columns=['id', 'aspect', 'input', 'start', 'end', 'sentiment'], index=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
out

,id,input,aspect,start,end,context,sentiment
0,13823,меню,Food,213,217,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",positive
1,13823,официантка,Service,242,252,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",positive
2,13823,удалилась,Service,270,279,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",positive
3,13823,ресторан,Whole,337,345,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",positive
4,13823,заведения,Whole,431,440,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",positive
...,...,...,...,...,...,...,...
681,11770,блюда,Food,267,272,"Для меня же минус был в том, что сами блюда сл...",positive
682,11770,ягод,Food,310,314,С добавлением ягод или фрукта и слишком много ...,positive
683,11770,фрукта,Food,319,325,С добавлением ягод или фрукта и слишком много ...,positive
684,11770,специй,Food,342,348,С добавлением ягод или фрукта и слишком много ...,positive


### 5. Классификатор 2. Способ 1.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

CATEGORIES = ['Whole', 'Interior', 'Service', 'Food', 'Price']

model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
  model.cuda()

def get_sentiment(text, return_type='label'):
    """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba

def score(proba):
  neg, neut, pos = proba
  if abs(neg - pos) < 0.25:
    return 'both'
  elif abs(neg - neut) < 0.1 and abs(pos - neut) < 0.1:
    return 'both'
  else:
    return model.config.id2label[proba.argmax()]

def get_full_sentiment(text, idx, df_ready):
    pred_cats = list(df_ready.loc[df_ready['id'] == idx]['aspect'])

    for c in CATEGORIES:
        if c not in pred_cats:
            s = 'absence'
        else:
            s = df_ready.loc[(df_ready['id'] == idx) & (df_ready['aspect'] == c)]['sent_pred'].item()
        yield c, s

In [ ]:
def category_sentiment(df):

  model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
  if torch.cuda.is_available():
      model.cuda()

  df_ready = df.groupby(['id', 'aspect']).agg({'context': lambda x: ' '.join(x)}).reset_index()
  df_ready['sent_pred'] = df_ready['context'].apply(lambda x: score(get_sentiment(x, 'proba')))

  return df_ready

df_ready = category_sentiment(out)

In [ ]:
df_ready

,id,aspect,context,sent_pred
0,785,Food,"Что касается торта , то он был очень вкусным ,...",both
1,785,Interior,"Ресторан понравился, интерьер приятный, много ...",positive
2,785,Service,"Что касается торта , то он был очень вкусным ,...",both
3,785,Whole,"Ресторан понравился, интерьер приятный, много ...",positive
4,797,Food,"Один раз днем, все понравилось,второй раз вече...",neutral
...,...,...,...,...
233,37819,Service,"Очень понравилась шоу программа и ведущая, кот...",positive
234,37819,Whole,Очень долго выбирали ресторан на Новогодний ка...,positive
235,38299,Interior,"Оказалось не зря... Очень уютная обстановка, н...",positive
236,38299,Service,"Оказалось не зря... Очень уютная обстановка, н...",both


In [ ]:
dev_texts = pd.read_csv('https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt', delimiter='\t', header=None, names=['text_id', 'text'])

with open('dev_pred_cats.txt', 'w') as f:
  for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
    for c, s in get_full_sentiment(text, idx, df_ready):
      print(idx, c, s, sep="\t", file=f)

### 6. Классификатор 2. Способ 2.

In [ ]:
def get_full_sentiment(text, idx, df_ready):
    pred_cats = list(df_ready.loc[df_ready['id'] == idx]['aspect'])

    for c in CATEGORIES:
        if c not in pred_cats:
            s = 'absence'
        else:
            s = df_ready.loc[(df_ready['id'] == idx) & (df_ready['aspect'] == c)]['sentiment'].item()
        yield c, s

def category_sentiment(df):
  df_total = pd.DataFrame(df.groupby(['id', 'aspect'])['sentiment'].agg(lambda x:x.value_counts().index[0])).reset_index()
  return df_total

In [ ]:
df_total = category_sentiment(out)

with open('dev_pred_cats_2.txt', 'w') as f:
  for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
    for c, s in get_full_sentiment(text, idx, df_total):
      print(idx, c, s, sep="\t", file=f)

###7. Оценка качества

####Baseline 1

In [ ]:
gold_test_path = "dev_aspects.txt"
pred_test_path = "dev_pred_aspects.txt"

gold_aspect_cats = {}
with open(gold_test_path) as fg:
    for line in fg:
        line = line.rstrip('\r\n').split('\t')
        if line[0] not in gold_aspect_cats:
            gold_aspect_cats[line[0]] = {"starts":[], "ends":[], "cats":[], "sents":[]}
        gold_aspect_cats[line[0]]["starts"].append(int(line[3]))
        gold_aspect_cats[line[0]]["ends"].append(int(line[4]))
        gold_aspect_cats[line[0]]["cats"].append(line[1])
        gold_aspect_cats[line[0]]["sents"].append(line[5])

full_match, partial_match, full_cat_match, partial_cat_match = 0, 0, 0, 0
total = 0
fully_matched_pairs = []
partially_matched_pairs = []
with open(pred_test_path) as fp:
    for line in fp:
        total += 1
        line = line.rstrip('\r\n').split('\t')
        start, end = int(line[3]), int(line[4])
        category = line[1]
        doc_gold_aspect_cats = gold_aspect_cats[line[0]]
        if start in doc_gold_aspect_cats["starts"]:
            i = doc_gold_aspect_cats["starts"].index(start)
            if doc_gold_aspect_cats["ends"][i] == end:
                full_match += 1
                if doc_gold_aspect_cats["cats"][i] == category:
                    full_cat_match += 1
                else:
                    partial_cat_match += 1
                fully_matched_pairs.append(
                    (
                        [
                            doc_gold_aspect_cats["starts"][i],
                            doc_gold_aspect_cats["ends"][i],
                            doc_gold_aspect_cats["cats"][i],
                            doc_gold_aspect_cats["sents"][i]
                        ],
                        line
                    )
                )
                continue
        for s_pos in doc_gold_aspect_cats["starts"]:
            if start <= s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if doc_gold_aspect_cats["ends"][i] == end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i],
                                doc_gold_aspect_cats["ends"][i],
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    continue
                matched = False
                for e_pos in doc_gold_aspect_cats["ends"][i:]:
                    if s_pos <= end <= e_pos:
                        partial_match += 1
                        partially_matched_pairs.append(
                            (
                                [
                                    doc_gold_aspect_cats["starts"][i],
                                    doc_gold_aspect_cats["ends"][i],
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                ],
                                line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        matched = True
                        break
                if matched:
                    break
            if start > s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if start < doc_gold_aspect_cats["ends"][i] <= end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i],
                                doc_gold_aspect_cats["ends"][i],
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    break

gold_size = sum([len(gold_aspect_cats[x]["cats"]) for x in gold_aspect_cats])
print(f"""
Full match precision: {full_match / total}
Full match recall: {full_match / gold_size}
Partial match ratio in pred: {(full_match + partial_match)  / total}
Full category accuracy: {full_cat_match / total}
Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
""")


Full match precision: 0.7327188940092166
Full match recall: 0.4008403361344538
Partial match ratio in pred: 0.8786482334869432
Full category accuracy: 0.7081413210445469
Partial category accuracy: 0.8740399385560675



Ср. базовое решение:

```
Full match precision: 0.48
Full match recall: 0.7159663865546219
Partial match ratio in pred: 0.6197183098591549
Full category accuracy: 0.46422535211267607
Partial category accuracy: 0.6033802816901408
```




####Baseline 2

In [ ]:
def sentiment_accuracy(matches):
    matched_sentiment = 0.
    for pair in matches:
        *_, gold_s = pair[0]
        *_, pred_s = pair[1]
        if gold_s == pred_s:
            matched_sentiment += 1
    print(f"Mention sentiment accuracy: {matched_sentiment / len(matches)}")

sentiment_accuracy(fully_matched_pairs)
sentiment_accuracy(partially_matched_pairs)

Mention sentiment accuracy: 0.7085953878406709
Mention sentiment accuracy: 0.7052631578947368


Ср. базовое решение:



```
Mention sentiment accuracy: 0.6772300469483568
Mention sentiment accuracy: 0.5720670391061452
```



####Baseline 3


**Способ 1**

In [ ]:
gold_test_cats_path = "dev_cats.txt"
pred_test_cats_path = "dev_pred_cats.txt"

with open(gold_test_cats_path) as gc, open(pred_test_cats_path) as pc:
    gold_labels = set(gc.readlines())
    pred_labels = set(pc.readlines())
    print(
        "Overall sentiment accuracy:",
        len(gold_labels & pred_labels) / len(gold_labels)
    )

Overall sentiment accuracy: 0.5464788732394367


**Способ 2**

In [ ]:
gold_test_cats_path = "dev_cats.txt"
pred_test_cats_path = "dev_pred_cats_2.txt"

with open(gold_test_cats_path) as gc, open(pred_test_cats_path) as pc:
    gold_labels = set(gc.readlines())
    pred_labels = set(pc.readlines())
    print(
        "Overall sentiment accuracy:",
        len(gold_labels & pred_labels) / len(gold_labels)
    )

Overall sentiment accuracy: 0.6450704225352113


Ср. базовое решение:


```
Overall sentiment accuracy: 0.523943661971831
```

